#Dataset

In [ ]:
TRAIN_DATA=

In [ ]:
TEST=

#code


In [ ]:
# Load pre-existing spacy model
import spacy
nlp = spacy.load("en_core_web_sm", disable=["lemmatizer"])
# Getting the pipeline component
ner=nlp.get_pipe("ner")

In [ ]:
# Adding labels to the `ner`
for _, annotations in TRAIN_DATA:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

In [ ]:
# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [ ]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(30):

    # shuufling examples  before every iteration
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
        print("Losses", losses)

Losses {'ner': 130.37268435955048}
Losses {'ner': 236.8420752286911}
Losses {'ner': 352.0752590894699}
Losses {'ner': 487.73580634593964}
Losses {'ner': 599.8777669668198}
Losses {'ner': 758.275307059288}
Losses {'ner': 902.6932066679001}
Losses {'ner': 988.472078204155}
Losses {'ner': 1139.740310549736}
Losses {'ner': 1274.196305155754}
Losses {'ner': 1413.106230378151}
Losses {'ner': 1539.2843886613846}
Losses {'ner': 1616.3227492570877}
Losses {'ner': 1767.5088058710098}
Losses {'ner': 1832.5191849470139}
Losses {'ner': 92.86623287200928}
Losses {'ner': 210.79034042358398}
Losses {'ner': 326.3977847099304}
Losses {'ner': 435.1685049533844}
Losses {'ner': 559.2632358074188}
Losses {'ner': 690.857507944107}
Losses {'ner': 801.8003146648407}
Losses {'ner': 919.8414151668549}
Losses {'ner': 1054.2494633197784}
Losses {'ner': 1121.3151636123657}
Losses {'ner': 1187.0285668373108}
Losses {'ner': 1316.6431965827942}
Losses {'ner': 1402.4291043281555}
Losses {'ner': 1515.9223880767822}
Loss

In [ ]:
# Testing the model
doc = nlp('اريد ان اعرف اذا لم ياخذ الطفل جرعة من جرعات تطعيم شلل الاطفال الفموي يقدم التطعيم الفموي على عدة مراحل عمرية يسبب له مشاكل او اية اعراض')
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('من', 'GR'), ('تطعيم', 'IV'), ('التطعيم', 'IV'), ('على', 'GR'), ('مراحل', 'N'), ('عمرية', 'N'), ('اعراض', 'N')]


#test

In [ ]:
def report_scores(scores, e):
    """
    prints precision recall and f_measure
    :param scores:
    :return:
    """

    precision = '%.2f' % scores['ents_p']
    recall = '%.2f' % scores['ents_r']
    f_measure = '%.2f' % scores['ents_f']
    print('%-25s %-10s %-10s %-10s' % (e, precision, recall, f_measure))

In [ ]:
import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer

def evaluate(nlp, examples, ent= ['VAC','M','IM','IV','LEN','IS','D','ID','IAG','T','IP','P','AG','S','DI','DO','F','DIS']):
    scorer = Scorer()
    for input_, annot in examples:
        text_entities = []
        for entity in annot.get('entities'):
            if ent in entity:
                text_entities.append(entity)
        doc_gold_text = nlp.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=text_entities)
        pred_value = nlp(input_)
        scorer.score(pred_value, gold)
    return scorer.scores


examples = TRAIN_DATA

#nlp = spacy.load('en_core_web_sm')
results = evaluate(nlp, examples)
print(results)

{'uas': 0.0, 'las': 0.0, 'las_per_type': {'compound': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'amod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'root': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'prep': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'pobj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nummod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'dobj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'advmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nsubj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'ccomp': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'acomp': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'npadvmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'aux': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'relcl': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'conj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'xcomp': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'intj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'appos': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'advcl': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'dative': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'attr': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'det': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'oprd': {'p': 0.0, 

In [ ]:
ents = ['VAC','M','IM','IV','LEN','IS','D','ID','IAG','T','IP','P','AG','S','DI','DO','F','DIS']
print('%-25s %-10s %-10s %-10s' % ("Entity", "Precision", "Recall", "f-measure"))
for e in ents:
    results = evaluate(nlp, TEST, e)
# calculate precision, recall, f_measure
    report_scores(results, e)

Entity                    Precision  Recall     f-measure 
VAC                       0.00       0.00       0.00      
M                         0.00       0.00       0.00      
IM                        0.00       0.00       0.00      
IV                        1.88       42.11      3.60      
LEN                       0.00       0.00       0.00      
IS                        0.00       0.00       0.00      
D                         0.00       0.00       0.00      
ID                        0.00       0.00       0.00      
IAG                       0.47       22.22      0.92      
T                         0.00       0.00       0.00      
IP                        0.00       0.00       0.00      
P                         0.00       0.00       0.00      
AG                        0.94       28.57      1.82      
S                         0.00       0.00       0.00      
DI                        0.00       0.00       0.00      
DO                        0.00       0.00       0.00    

#SAVING AND LOADING

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Save the  model to directory
#output_dir = Path('/content/')

output_dir = Path('/content/gdrive/MyDrive/NER_BERT/')

#/content/drive/MyDrive/NER_BERT

nlp.to_disk(output_dir)
print("Saved model to", output_dir)

# Load the saved model and predict
print("Loading from", output_dir)
nlp_updated = spacy.load(output_dir)



doc = nlp_updated("Fridge can be ordered in FlipKart" )
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Saved model to /content/gdrive/MyDrive/NER_BERT
Loading from /content/gdrive/MyDrive/NER_BERT
Entities [('ordered', 'V')]


In [ ]:
#nlp.to_disk('nlp_spacyy_model_drug')
#####nlp_model = spacy.load('nlp_spacyy_model_drug')

In [ ]:
#doc=nlp_model("و قالبستي كلاااام")
#####print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
#joblib.dump(value=sgd, filename='sgd_model.pkl', protocol=2)

In [ ]:
import pickle

pickle.dump(nlp, open( "nlp.p", "wb" ))

In [ ]:
#file = open("nlp.p",'r')
#object_file = pickle.load(file)

In [ ]:
from sklearn.externals import joblib
# Serialize the pipeline
joblib.dump(value=nlp, filename='spa22_model_vacc.pkl', protocol=2)

new_model = joblib.load('spa22_model_vacc.pkl')

In [ ]:
doc=new_model("و قالبستي كلاااام")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities []


In [ ]:
from spacy.lemmatizer import Lemmatizer

In [ ]:
spacyy= joblib.load('spa22_model_vacc.pkl')

In [ ]:
doc=spacyy('اريد ان اعرف اذا لم ياخذ الطفل جرعة من جرعات تطعيم شلل الاطفال الفموي يقدم التطعيم الفموي على عدة مراحل عمرية يسبب له مشاكل او اية اعراض')
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('من', 'GR'), ('تطعيم', 'IV'), ('التطعيم', 'IV'), ('على', 'GR'), ('مراحل', 'N'), ('عمرية', 'N'), ('اعراض', 'N')]
